# Script para limpar os discursos da base:
- remover stopwords
- remover pontuação
- remover caracteres especiais
- remover palavras/termos muito comuns, por exemplo, "Sr. Presidente"

In [1]:
import os
import pandas as pd
import re

import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
from unicodedata import normalize


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:

def remove_stop_words(data, list_stop_words = None):
    """
    Função para remover stopwords em português
    """
    print('  Remove Stop Words')

    #list_stop_words = set(stopwords.words('portuguese'))
    if list_stop_words != None:
        output = []
        for sentence in data:
            temp_list = []
            for word in sentence.split():
                if word.lower() not in list_stop_words:
                    temp_list.append(word)
            output.append(' '.join(temp_list))

        data = pd.Series(output)    
    
    return data

def remove_punct(data):
    """
    Função para remover pontuações
    """
    print('  Remove Punction')

    puncts = ["\\" +s for s in string.punctuation]
    puncts = '|'.join(puncts)
    data = data.replace(puncts,' ',regex=True)    
    return data

def remove_special_caract(array_data):
    """
    Função para substituir caracteres especiais
    """
    print('  Remove Special Caract')
    data = [normalize('NFKD', data).encode('ASCII', 'ignore').decode('ASCII') for data in array_data]
    return data

def remove_patter_terms(data):
    """
    Função para remover orações que há em todos os discursos, como "senhor presidente", "senhora(s) deputadas"
    """
    pattern = "^(.+?)-\sSr|(((O?\s?SR\.?\s+?)|(A?\s?SRA\.?\s+?))(\s+DEPUTADO|\s+DEPUTADA|\s+PRESIDENTE|\s+PRESIDENTA)?)|\
              (SR\.?\s+?)|(SRA\.?\s+?)|srs|sras|(presidente|deputado(s)|deputada(s))"
    
    data = re.sub(pattern, '',data, flags = re.IGNORECASE)
    
    return data

def clean_text(data):
    """
    Função para fazer pre-processamento dos textos
    """
    print('Cleaning text...')
    
    data =  data.apply(remove_patter_terms)
    
    data = remove_punct(data)
    data = data.str.lower()
    
    list_stop_words = remove_special_caract(stopwords.words('portuguese'))
    list_stop_words.extend(['ser','ja','ha','exa','ainda','ate','todo','todos','toda','todas','devem'])
    list_stop_words = set(list_stop_words)
    data = remove_stop_words(data, list_stop_words)
    
    data = data.replace('\s+',' ',regex=True)
        
    return data

In [7]:
directory = "../data/csvs/"
discursos = pd.read_csv(directory+'discursos_plen.csv')

In [8]:
# retirando caracteres especiais das stopwords, porque o texto está sem!!
list_stop_words = remove_special_caract(stopwords.words('portuguese'))

# inserindo mais palavras pra ser removidas
list_stop_words.extend(['ser','ja','ha','exa','ainda','ate','todo','todos','toda','todas','devem'])

  Remove Special Caract


In [9]:
discursos.loc[:,'CLEAN_CONTENT'] = clean_text(discursos.loc[:,'CLEAN_CONTENT'])


Cleaning text...
  Remove Punction
  Remove Special Caract
  Remove Stop Words


In [10]:
## salvando dados limpos em uma nova base
discursos.to_csv(directory+'discursos_plen_limpo.csv', index = False)